In [1]:
library(coldpool)
library(akgfmaps)

NameError: name 'library' is not defined

In [4]:
proj_crs <- coldpool:::ebs_proj_crs

ebs_nep_csv_path <- '/work/Utheri.Wagura/NEP/plotting/Figure_19_20/index_hauls_temperature_data_nep.csv'

### Calculate Cold Pool Area for NEP tob

In [5]:
# Interpolate gear temperature and write rasters for SEBS - nep bottom temp values
interpolation_wrapper(temp_data_path = ebs_nep_csv_path,
                      proj_crs = proj_crs,
                      cell_resolution = 5000, # 5x5 km grid resolution
                      select_years = 1993:2024,
                      interp_variable = "nep_tob",
                      select_region = "sebs",
                      methods = "Ste") 

In [ ]:
# Calculate cold pool area for NEP
bottom_temp_files <- list.files(here::here("output", "raster", "sebs", "nep_tob"), 
                                full.names = TRUE,
                                pattern = "ste_")

bt_df <- data.frame(YEAR = numeric(length = length(bottom_temp_files)),
                    AREA_LTE2_KM2 = numeric(length = length(bottom_temp_files)),
                    AREA_LTE1_KM2 = numeric(length = length(bottom_temp_files)),
                    AREA_LTE0_KM2 = numeric(length = length(bottom_temp_files)),
                    AREA_LTEMINUS1_KM2 = numeric(length = length(bottom_temp_files)),
                    MEAN_GEAR_TEMPERATURE = numeric(length = length(bottom_temp_files)),
                    MEAN_BT_LT100M = numeric(length = length(bottom_temp_files)))

# Setup mask to calculate mean bottom temperature from <100 m strata
ebs_layers <- akgfmaps::get_base_layers(select.region = "sebs", set.crs = proj_crs)

lt100_strata <- ebs_layers$survey.strata |>
  dplyr::filter(Stratum %in% c(10, 20, 31, 32, 41, 42, 43)) |>
  dplyr::group_by(SURVEY) |>
  dplyr::summarise()

for(i in 1:length(bottom_temp_files)) {
  bt_raster <- terra::rast(bottom_temp_files[i])
  bt_df$YEAR[i] <- as.numeric(gsub("[^0-9.-]", "", names(bt_raster))) # Extract year
  bt_df$AREA_LTE2_KM2[i] <- bt_raster |> 
    cpa_from_raster(raster_units = "m", temperature_threshold = 2)
  bt_df$AREA_LTE1_KM2[i] <- bt_raster |> 
    cpa_from_raster(raster_units = "m", temperature_threshold = 1)
  bt_df$AREA_LTE0_KM2[i] <- bt_raster |> 
    cpa_from_raster(raster_units = "m", temperature_threshold = 0)
  bt_df$AREA_LTEMINUS1_KM2[i] <- bt_raster |> 
    cpa_from_raster(raster_units = "m", temperature_threshold = -1)
  bt_df$MEAN_GEAR_TEMPERATURE[i] <- mean(terra::values(bt_raster), na.rm = TRUE)
  lt100_temp <- terra::mask(bt_raster, 
                            lt100_strata,
                            touches = FALSE)
  bt_df$MEAN_BT_LT100M[i] <- mean(terra::values(lt100_temp), na.rm = TRUE) 
  
}

write.csv(bt_df, "nep_cpa.csv", row.names=FALSE)

### Calculate Cold Pool Area for AFSC Trawl Gear Temperature

In [ ]:
# Interpolate gear temperature and write rasters for SEBS
interpolation_wrapper(temp_data_path = ebs_nep_csv_path,
                      proj_crs = proj_crs,
                      cell_resolution = 5000, # 5x5 km grid resolution
                      select_years = 1993:2024,
                      interp_variable = "gear_temperature",
                      select_region = "sebs",
                      methods = "Ste") 

In [ ]:
# Calculate cold pool area
bottom_temp_files <- list.files(here::here("output", "raster", "sebs", "gear_temperature"), 
                                full.names = TRUE,
                                pattern = "ste_")

bt_df <gtk- data.frame(YEAR = numeric(length = length(bottom_temp_files)),
                    AREA_LTE2_KM2 = numeric(length = length(bottom_temp_files)),
                    AREA_LTE1_KM2 = numeric(length = length(bottom_temp_files)),
                    AREA_LTE0_KM2 = numeric(length = length(bottom_temp_files)),
                    AREA_LTEMINUS1_KM2 = numeric(length = length(bottom_temp_files)),
                    MEAN_GEAR_TEMPERATURE = numeric(length = length(bottom_temp_files)),
                    MEAN_BT_LT100M = numeric(length = length(bottom_temp_files)))

# Setup mask to calculate mean bottom temperature from <100 m strata
ebs_layers <- akgfmaps::get_base_layers(select.region = "sebs", set.crs = proj_crs)

lt100_strata <- ebs_layers$survey.strata |>
  dplyr::filter(Stratum %in% c(10, 20, 31, 32, 41, 42, 43)) |>
  dplyr::group_by(SURVEY) |>
  dplyr::summarise()

for(i in 1:length(bottom_temp_files)) {
  bt_raster <- terra::rast(bottom_temp_files[i])
  bt_df$YEAR[i] <- as.numeric(gsub("[^0-9.-]", "", names(bt_raster))) # Extract year
  bt_df$AREA_LTE2_KM2[i] <- bt_raster |> 
    cpa_from_raster(raster_units = "m", temperature_threshold = 2)
  bt_df$AREA_LTE1_KM2[i] <- bt_raster |> 
    cpa_from_raster(raster_units = "m", temperature_threshold = 1)
  bt_df$AREA_LTE0_KM2[i] <- bt_raster |> 
    cpa_from_raster(raster_units = "m", temperature_threshold = 0)
  bt_df$AREA_LTEMINUS1_KM2[i] <- bt_raster |> 
    cpa_from_raster(raster_units = "m", temperature_threshold = -1)
  bt_df$MEAN_GEAR_TEMPERATURE[i] <- mean(terra::values(bt_raster), na.rm = TRUE)
  lt100_temp <- terra::mask(bt_raster, 
                            lt100_strata,
                            touches = FALSE)
  bt_df$MEAN_BT_LT100M[i] <- mean(terra::values(lt100_temp), na.rm = TRUE) 
  
}

write.csv(bt_df, "trawl_cpa.csv", row.names=FALSE)